## STRAICO

In [ ]:
import json
import csv
import os
import requests
from typing import List, Optional, Dict, Any
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from functools import partial
import time

# API endpoint
API_URL = "https://api.straico.com/v1/prompt/completion"

# Hardcoded API key and default model(s)
API_KEY = "v8-Ps43kVOZynO8LHli5ay9oXNDR7VNMmDkxONui5xcp8Q0tjAo"

PROMPT_ID_MAP = {
    "prompt_1": "P1", # Dada la palabra --> Definición del modismo.
    "prompt_2": "P2", # Dada la palabra --> Decir si es modismo o no.
    "prompt_3": "P3", # Dado el ejemplo --> Reemplazo literal y definición del remplazo.
}

# Configuración de paralelización
MAX_WORKERS = 8  # Puedes aumentar a 10-12 si el API lo permite

# Lock global para operaciones de escritura thread-safe
_write_lock = threading.Lock()

In [ ]:
def send_prompt(message: str, models: Optional[List[str]] = None) -> Dict[str, Any]:
    
    """ Send a single text prompt to Straico and return the parsed response.
        This function always performs a live HTTP request.
    """

    payload: Dict[str, Any] = {"models": models, "message": message}
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

    try:
        resp = requests.post(API_URL, headers=headers, json=payload, timeout=30)
    except requests.exceptions.RequestException as exc:
        return {"error": str(exc)}

    # Parse JSON if available
    try:
        data = resp.json()
    except Exception:
        data = None

    if 200 <= resp.status_code < 300:

        # Try to extract the assistant 'content' text from the common response
        # shape: data -> completions -> <model> -> completion -> choices[0] -> message -> content

        if isinstance(data, dict):
            completions = data.get('data', {}).get('completions', {})

            # If we passed a single model, try to use that key; otherwise take first
            model_key = None

            if models and len(models) == 1:
                model_key = models[0]
            if not model_key and isinstance(completions, dict) and len(completions) > 0:
                model_key = next(iter(completions.keys()))

            if model_key and model_key in completions:
                try:
                    content = completions[model_key]['completion']['choices'][0]['message']['content']
                    return content
                
                except Exception:
                    pass

        # Fallback: return raw text or the full JSON string
        if data is None:
            return resp.text
        
        try:
            return json.dumps(data, ensure_ascii=False)
        except Exception:
            return str(data)

    return {"error": f"status={resp.status_code}", "response": data if data is not None else resp.text}


### Configuración General

In [ ]:
def _import_prompts() -> Dict[str, str]:
    """Load prompt_1..prompt_3 from prompts.py and return as a dict.
    If prompts.py isn't present, returns an empty dict.
    """
    try:
        from Straico import prompts as p  # type: ignore
    except Exception:
        try:
            import prompts as p  # type: ignore
        except Exception:
            return {}

    out: Dict[str, str] = {}
    for name in ("prompt_1", "prompt_2", "prompt_3"):
        if hasattr(p, name):
            out[name] = getattr(p, name)
    return out

#### Modelos a usar

In [ ]:
# Cargar modelos desde el archivo text_model_ids.txt
MODELS_FILE = 'Straico/text_model_usefull.txt'

def load_models_from_file(filepath):
    """Carga los nombres de modelos desde un archivo de texto."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo {filepath}")
        return []

DEFAULT_MODELS = load_models_from_file(MODELS_FILE)[:20]

In [ ]:
DEFAULT_MODELS

#### Ejecusión

In [ ]:
# Configuración
DATASET_PATH = 'modismos_Dataset_Final.csv'
N_ROWS = 20
RESPONSES_DIR = 'Straico'

# Cargar prompts
PROMPTS = _import_prompts()
print(f"Prompts cargados: {list(PROMPTS.keys())}")

In [ ]:
def cargar_dataset(n_rows=None):
    """Carga el dataset y retorna una lista de diccionarios con modismo, significado y ejemplo."""
    rows = []
    seen_modismos = set()
    
    with open(DATASET_PATH, encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=';')
        for r in reader:
            modismo = r.get('modismo', '').strip()
            if not modismo or modismo.casefold() in seen_modismos:
                continue
            seen_modismos.add(modismo.casefold())
            
            rows.append({
                'modismo': modismo,
                'significado': r.get('significado', '').strip(),
                'ejemplo': r.get('ejemplo', '').strip()
            })
            
            if n_rows and len(rows) >= n_rows:
                break
    
    return rows


def sanitize_model_name(model_name):
    """Convierte nombres de modelos en nombres válidos para nombres de archivos."""
    return model_name.replace('/', '_').replace(':', '_').replace('-', '_').replace('.', '_')


def save_json(filepath, data):
    """Guarda datos en un archivo JSON de forma thread-safe."""
    with _write_lock:
        try:
            dir_path = os.path.dirname(filepath)
            if dir_path:
                os.makedirs(dir_path, exist_ok=True)
            
            # Escritura atómica usando archivo temporal
            temp_filepath = f"{filepath}.tmp"
            with open(temp_filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            
            # Renombrar es operación atómica en sistemas POSIX
            os.replace(temp_filepath, filepath)
            print(f"[OK] Guardado: {filepath}")
        except Exception as e:
            print(f"[ERROR] No se pudo guardar {filepath}: {e}")
            # Limpiar archivo temporal si existe
            if os.path.exists(temp_filepath):
                try:
                    os.remove(temp_filepath)
                except:
                    pass


def save_model_response(base_dir, prompt_name, model_name, data):
    """Guarda la respuesta de un modelo específico en su propia carpeta.
    
    Args:
        base_dir: Directorio base (ej: 'Straico')
        prompt_name: Nombre del prompt (ej: 'Prompt 1')
        model_name: Nombre del modelo
        data: Lista de respuestas del modelo
    """
    model_safe_name = sanitize_model_name(model_name)
    model_dir = os.path.join(base_dir, prompt_name, model_safe_name)
    filepath = os.path.join(model_dir, f"{model_safe_name}.json")
    save_json(filepath, data)


def save_consolidated_response(base_dir, prompt_name, all_models_data):
    """Guarda todas las respuestas consolidadas en un solo archivo.
    
    Args:
        base_dir: Directorio base (ej: 'Straico')
        prompt_name: Nombre del prompt (ej: 'Prompt 1')
        all_models_data: Diccionario con {model_name: [respuestas]}
    """
    filepath = os.path.join(base_dir, prompt_name, "all_models.json")
    save_json(filepath, all_models_data)


def save_partial_progress(base_dir, prompt_name, progress_data):
    """Guarda el progreso parcial durante la ejecución.
    
    Args:
        base_dir: Directorio base (ej: 'Straico')
        prompt_name: Nombre del prompt (ej: 'Prompt 1')
        progress_data: Datos parciales acumulados
    """
    filepath = os.path.join(base_dir, prompt_name, "progress.json")
    save_json(filepath, progress_data)

---
## PROMPT 1: Modismo → Definición

**Input**: `modismo`  
**Output**: `definicion`  
**Guardado**: 
- `Straico/Prompt 1/{modelo}/{modelo}.json` (por cada modelo)
- `Straico/Prompt 1/all_models.json` (consolidado)
- `Straico/Prompt 1/progress.json` (progreso parcial)

In [ ]:
def process_single_model_prompt_1(model, dataset, template, responses_dir="Straico"):
    """
    Procesa un solo modelo para el Prompt 1.
    Esta función se ejecutará en paralelo para múltiples modelos.
    """
    model_responses = []
    errors_count = 0
    
    for row in tqdm(dataset, desc=f"[{model}]", position=None, leave=True):
        try:
            modismo = row.get('modismo', '').strip()
            if not modismo:
                continue
            
            # Armar el prompt
            prompt_text = template.replace('{{modismo}}', modismo)
            
            # Obtener respuesta del modelo con retry básico
            max_retries = 3
            resp = None
            for attempt in range(max_retries):
                resp = send_prompt(prompt_text, models=[model])
                if not isinstance(resp, dict) or 'error' not in resp:
                    break
                if attempt < max_retries - 1:
                    time.sleep(1 * (attempt + 1))  # Backoff exponencial
            
            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                if 'error' in resp:
                    errors_count += 1
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}
            
            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "model": model,
                "response": response_data
            }
            
            model_responses.append(entry)
            
        except Exception as e:
            print(f"[ERROR] Modelo {model}, modismo '{modismo}': {e}")
            errors_count += 1
            continue
    
    # Guardar respuestas del modelo
    try:
        save_model_response(responses_dir, "Prompt 1", model, model_responses)
    except Exception as e:
        print(f"[ERROR] No se pudo guardar respuestas del modelo {model}: {e}")
    
    if errors_count > 0:
        print(f"[WARNING] Modelo {model}: {errors_count} errores encontrados")
    
    return model, model_responses


def run_prompt_1(models=DEFAULT_MODELS, n_rows=N_ROWS, max_workers=MAX_WORKERS):
    """
    PROMPT 1: Dada la palabra/modismo -> Generar definicion (PARALELIZADO)
    INPUT: modismo
    OUTPUT: definicion
    
    Optimizado para Mac M4 Pro con procesamiento paralelo de modelos.
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 1: Modismo -> Definicion (PARALELO)")
    print("=" * 80)

    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    if not dataset:
        print("[ERROR] No se pudo cargar el dataset")
        return
    
    print(f"Dataset cargado: {len(dataset)} filas")
    print(f"Modelos a consultar: {len(models)}")
    print(f"Workers paralelos: {max_workers}")

    # Obtener template del prompt
    template = PROMPTS.get('prompt_1')
    if not template:
        print("[ERROR] prompt_1 no encontrado")
        return

    all_models_data = {}
    completed_models = 0
    failed_models = 0
    
    # Procesar modelos en paralelo
    print("\n[INFO] Iniciando procesamiento paralelo...")
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Crear función parcial con parámetros fijos
        process_func = partial(process_single_model_prompt_1, 
                              dataset=dataset, 
                              template=template, 
                              responses_dir=RESPONSES_DIR)
        
        # Enviar todos los modelos a procesar
        future_to_model = {executor.submit(process_func, model): model for model in models}
        
        # Recolectar resultados conforme van completando
        for future in as_completed(future_to_model):
            model = future_to_model[future]
            try:
                model_name, model_responses = future.result(timeout=600)  # Timeout 10 min por modelo
                all_models_data[model_name] = model_responses
                completed_models += 1
                print(f"[OK] Completado {completed_models}/{len(models)}: {model_name} ({len(model_responses)} respuestas)")
            except Exception as exc:
                print(f"[ERROR] Fallo en {model}: {exc}")
                all_models_data[model] = []
                failed_models += 1

    elapsed_time = time.time() - start_time
    
    # Guardar consolidado final
    print(f"\n[INFO] Guardando archivo consolidado...")
    try:
        save_consolidated_response(RESPONSES_DIR, "Prompt 1", all_models_data)
        print(f"[OK] Archivo consolidado guardado correctamente")
    except Exception as e:
        print(f"[ERROR] No se pudo guardar archivo consolidado: {e}")

    print("\n" + "=" * 80)
    print(f"PROMPT 1 COMPLETADO")
    print(f"Modelos exitosos: {completed_models}/{len(models)}")
    print(f"Modelos fallidos: {failed_models}/{len(models)}")
    print(f"Tiempo total: {elapsed_time:.2f} segundos")
    print("=" * 80)

In [ ]:
# Ejecutar Prompt 1
run_prompt_1()

---
## PROMPT 2: Modismo → Es Modismo (Sí/No)

**Input**: `modismo`  
**Output**: `es_modismo` (Sí/No)  
**Guardado**: 
- `Straico/Prompt 2/{modelo}/{modelo}.json` (por cada modelo)
- `Straico/Prompt 2/all_models.json` (consolidado)
- `Straico/Prompt 2/progress.json` (progreso parcial)

In [ ]:
def process_single_model_prompt_2(model, dataset, template, responses_dir="Straico"):
    """
    Procesa un solo modelo para el Prompt 2.
    Esta función se ejecutará en paralelo para múltiples modelos.
    """
    model_responses = []
    errors_count = 0
    
    for row in tqdm(dataset, desc=f"[{model}]", position=None, leave=True):
        try:
            modismo = row.get('modismo', '').strip()
            if not modismo:
                continue
            
            # Armar el prompt
            prompt_text = template.replace('{{modismo}}', modismo)
            
            # Obtener respuesta del modelo con retry básico
            max_retries = 3
            resp = None
            for attempt in range(max_retries):
                resp = send_prompt(prompt_text, models=[model])
                if not isinstance(resp, dict) or 'error' not in resp:
                    break
                if attempt < max_retries - 1:
                    time.sleep(1 * (attempt + 1))
            
            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                if 'error' in resp:
                    errors_count += 1
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}
            
            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "model": model,
                "response": response_data
            }
            
            model_responses.append(entry)
            
        except Exception as e:
            print(f"[ERROR] Modelo {model}, modismo '{modismo}': {e}")
            errors_count += 1
            continue
    
    # Guardar respuestas del modelo
    try:
        save_model_response(responses_dir, "Prompt 2", model, model_responses)
    except Exception as e:
        print(f"[ERROR] No se pudo guardar respuestas del modelo {model}: {e}")
    
    if errors_count > 0:
        print(f"[WARNING] Modelo {model}: {errors_count} errores encontrados")
    
    return model, model_responses


def run_prompt_2(models=DEFAULT_MODELS, n_rows=N_ROWS, max_workers=MAX_WORKERS):
    """
    PROMPT 2: Dada la palabra/modismo -> Determinar si es modismo (Si/No) (PARALELIZADO)
    INPUT: modismo
    OUTPUT: es_modismo
    
    Optimizado para Mac M4 Pro con procesamiento paralelo de modelos.
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 2: Modismo -> Es Modismo (Si/No) (PARALELO)")
    print("=" * 80)

    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    if not dataset:
        print("[ERROR] No se pudo cargar el dataset")
        return
    
    print(f"Dataset cargado: {len(dataset)} filas")
    print(f"Modelos a consultar: {len(models)}")
    print(f"Workers paralelos: {max_workers}")

    # Obtener template del prompt
    template = PROMPTS.get('prompt_2')
    if not template:
        print("[ERROR] prompt_2 no encontrado")
        return

    all_models_data = {}
    completed_models = 0
    failed_models = 0
    
    # Procesar modelos en paralelo
    print("\n[INFO] Iniciando procesamiento paralelo...")
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Crear función parcial con parámetros fijos
        process_func = partial(process_single_model_prompt_2, 
                              dataset=dataset, 
                              template=template, 
                              responses_dir=RESPONSES_DIR)
        
        # Enviar todos los modelos a procesar
        future_to_model = {executor.submit(process_func, model): model for model in models}
        
        # Recolectar resultados conforme van completando
        for future in as_completed(future_to_model):
            model = future_to_model[future]
            try:
                model_name, model_responses = future.result(timeout=600)
                all_models_data[model_name] = model_responses
                completed_models += 1
                print(f"[OK] Completado {completed_models}/{len(models)}: {model_name} ({len(model_responses)} respuestas)")
            except Exception as exc:
                print(f"[ERROR] Fallo en {model}: {exc}")
                all_models_data[model] = []
                failed_models += 1

    elapsed_time = time.time() - start_time
    
    # Guardar consolidado final
    print(f"\n[INFO] Guardando archivo consolidado...")
    try:
        save_consolidated_response(RESPONSES_DIR, "Prompt 2", all_models_data)
        print(f"[OK] Archivo consolidado guardado correctamente")
    except Exception as e:
        print(f"[ERROR] No se pudo guardar archivo consolidado: {e}")

    print("\n" + "=" * 80)
    print(f"PROMPT 2 COMPLETADO")
    print(f"Modelos exitosos: {completed_models}/{len(models)}")
    print(f"Modelos fallidos: {failed_models}/{len(models)}")
    print(f"Tiempo total: {elapsed_time:.2f} segundos")
    print("=" * 80)

In [ ]:
# Ejecutar Prompt 2
run_prompt_2()

---
## PROMPT 3: Modismo + Ejemplo → Literal + Definición

**Input**: `modismo` + `ejemplo`  
**Output**: `literal` + `definicion`  
**Guardado**: 
- `Straico/Prompt 3/{modelo}/{modelo}.json` (por cada modelo)
- `Straico/Prompt 3/all_models.json` (consolidado)
- `Straico/Prompt 3/progress.json` (progreso parcial)

In [ ]:
def process_single_model_prompt_3(model, dataset, template, responses_dir="Straico"):
    """
    Procesa un solo modelo para el Prompt 3.
    Esta función se ejecutará en paralelo para múltiples modelos.
    """
    model_responses = []
    errors_count = 0
    
    for row in tqdm(dataset, desc=f"[{model}]", position=None, leave=True):
        try:
            modismo = row.get('modismo', '').strip()
            ejemplo = row.get('ejemplo', '').strip()
            
            if not modismo or not ejemplo:
                continue
            
            # Armar el prompt
            prompt_text = template.replace('{{modismo}}', modismo).replace('{{ejemplo}}', ejemplo)
            
            # Obtener respuesta del modelo con retry básico
            max_retries = 3
            resp = None
            for attempt in range(max_retries):
                resp = send_prompt(prompt_text, models=[model])
                if not isinstance(resp, dict) or 'error' not in resp:
                    break
                if attempt < max_retries - 1:
                    time.sleep(1 * (attempt + 1))
            
            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                if 'error' in resp:
                    errors_count += 1
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}
            
            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "ejemplo": ejemplo,
                "model": model,
                "response": response_data
            }
            
            model_responses.append(entry)
            
        except Exception as e:
            print(f"[ERROR] Modelo {model}, modismo '{modismo}': {e}")
            errors_count += 1
            continue
    
    # Guardar respuestas del modelo
    try:
        save_model_response(responses_dir, "Prompt 3", model, model_responses)
    except Exception as e:
        print(f"[ERROR] No se pudo guardar respuestas del modelo {model}: {e}")
    
    if errors_count > 0:
        print(f"[WARNING] Modelo {model}: {errors_count} errores encontrados")
    
    return model, model_responses


def run_prompt_3(models=DEFAULT_MODELS, n_rows=N_ROWS, max_workers=MAX_WORKERS):
    """
    PROMPT 3: Dado modismo + ejemplo -> Generar literal + definicion (PARALELIZADO)
    INPUT: modismo + ejemplo
    OUTPUT: literal + definicion
    
    Optimizado para Mac M4 Pro con procesamiento paralelo de modelos.
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 3: Modismo + Ejemplo -> Literal + Definicion (PARALELO)")
    print("=" * 80)

    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    if not dataset:
        print("[ERROR] No se pudo cargar el dataset")
        return
    
    dataset_with_examples = [row for row in dataset if row.get('ejemplo', '').strip()]
    print(f"Dataset cargado: {len(dataset_with_examples)} filas con ejemplo")
    print(f"Modelos a consultar: {len(models)}")
    print(f"Workers paralelos: {max_workers}")

    # Obtener template del prompt
    template = PROMPTS.get('prompt_3')
    if not template:
        print("[ERROR] prompt_3 no encontrado")
        return

    all_models_data = {}
    completed_models = 0
    failed_models = 0
    
    # Procesar modelos en paralelo
    print("\n[INFO] Iniciando procesamiento paralelo...")
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Crear función parcial con parámetros fijos
        process_func = partial(process_single_model_prompt_3, 
                              dataset=dataset_with_examples, 
                              template=template, 
                              responses_dir=RESPONSES_DIR)
        
        # Enviar todos los modelos a procesar
        future_to_model = {executor.submit(process_func, model): model for model in models}
        
        # Recolectar resultados conforme van completando
        for future in as_completed(future_to_model):
            model = future_to_model[future]
            try:
                model_name, model_responses = future.result(timeout=600)
                all_models_data[model_name] = model_responses
                completed_models += 1
                print(f"[OK] Completado {completed_models}/{len(models)}: {model_name} ({len(model_responses)} respuestas)")
            except Exception as exc:
                print(f"[ERROR] Fallo en {model}: {exc}")
                all_models_data[model] = []
                failed_models += 1

    elapsed_time = time.time() - start_time
    
    # Guardar consolidado final
    print(f"\n[INFO] Guardando archivo consolidado...")
    try:
        save_consolidated_response(RESPONSES_DIR, "Prompt 3", all_models_data)
        print(f"[OK] Archivo consolidado guardado correctamente")
    except Exception as e:
        print(f"[ERROR] No se pudo guardar archivo consolidado: {e}")

    print("\n" + "=" * 80)
    print(f"PROMPT 3 COMPLETADO")
    print(f"Modelos exitosos: {completed_models}/{len(models)}")
    print(f"Modelos fallidos: {failed_models}/{len(models)}")
    print(f"Tiempo total: {elapsed_time:.2f} segundos")
    print("=" * 80)

In [ ]:
# Ejecutar Prompt 3
run_prompt_3()